# Introduction to Topic Modeling
## Day 1: The Basics
---
---

## Learning goals

* Think through how and why you might use topic modeling in a text analysis project
* Implement a basic topic modeling algorithm and learn how to tweak it
* Understand the intuitions behind `Latent Dirichlet Allocation` (LDA) for topic modeling
* Get familiar with (some of) Python's versatile `scikit-learn` library
* Get familiar with the `Pandas` package and manipulating data
* Learn how to prepare a `Document-Term Matrix` (DTM), why this is useful, and what "bag of words" means


## Outline
- [The 'what' and 'why' of topic modeling](#whatwhy)
    - [How LDA works (briefly)](#mechanics)
- [How to join in with coding](#joinin)
- [The Pandas dataframe: Children's literature](#data)
    - [Explore the data](#explore)
- [Text preprocessing: The Document-Term Matrix (DTM)](#preprocess)
- [Train a topic model using LDA](#train)
- [Document-by-topic distribution](#topics)
- [LDA as dimensionality reduction](#dimensionality)


## Key terms
* *document-term matrix (DTM)*:
    * a mathematical matrix that describes the frequency of terms that occur in a collection of documents. In a DTM, rows correspond to documents in the collection and columns correspond to terms. Making a DTM is a common preprocessing step: it takes in tokenized texts and makes them ready for downstream tasks (e.g., topic models, supervised machine learning models).
* *topic modeling*:
    * A statistical model to uncover abstract topics within a text. It uses the co-occurrence fo words within documents, compared to their distribution across documents, to uncover these abstract themes. The output is a list of weighted words, which indicate the subject of each topic, and a weight distribution across topics for each document.
* *Latent Dirichlet Allocation (LDA)*:
    * A popular implementation of topic modeling that assumes a Dirichlet prior. It estimates a distribution of documents over topics and a distribution of words over topics. It does not take document order into account (unlike some others topic modeling algorithms).
* *unsupervised text analysis*:
    * Learning patterns or structure in text data through the associations of linguistic units (e.g., word co-associations). Uses include discovering latent clusters or dimensions, reducing dimensionality, and outlier detection.

## The 'what' and 'why' of topic modeling <a id='whatwhy'></a>

Topic modeling is a _distant reading_ technique for finding structure in large collections of text, without actually reading everything by eye. If you have hundreds or thousands of documents and want to understand roughly what your corpus contains, then topic modeling may be for you.

A topic modeling program finds the words that appear frequently together in a document and groups them together to form **topics**, which are mixtures of words that characterize the document's themes or underlying ideas. Topic modeling is a kind of _unsupervised text analysis_ that learns what topics characterize a corpus based on word co-associations within and across documents.

As a simple example, one topic of this [Wikipedia article on black holes](https://en.wikipedia.org/wiki/Black_hole) is:

* black, hole, mass, star

![First picture of a supermassive black hole, captured in 2019](https://upload.wikimedia.org/wikipedia/commons/thumb/c/cf/Black_hole_-_Messier_87.jpg/320px-Black_hole_-_Messier_87.jpg "First picture of a supermassive black hole, captured in 2019")

Not too surprising, but this topic does seem accurate on the face of it. What about a document that we are less familiar with? Here is a topic of a [speech made by John F. Kennedy at Rice University in 1962](https://er.jsc.nasa.gov/seh/ricetalk.htm):

* space, new, year, man

![Charles Conrad Jr., Apollo 12 Commander, examines the unmanned Surveyor III spacecraft on the Moon](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/Surveyor_3-Apollo_12.jpg/274px-Surveyor_3-Apollo_12.jpg "Charles Conrad Jr., Apollo 12 Commander, examines the unmanned Surveyor III spacecraft on the Moon")

This is Kennedy's famous 'we choose to go to the moon' speech. Notice that 'moon' is not in this topic; but the speech does cover the history of humankind's ("man's") endeavours and emphasises a forward-looking perspective (the "new"-ness of advancements).

As suggested by the simplified examples above, human intervention is still required to interpret what topics might 'mean'. Topic modeling is not magic; it is a tool that requires informed use and careful review, just like any other. Ideally, when interpreting topics we consider their frequent and distinctive words as well as specific texts high in topics of interest. But just a few frequent words by topic can tell us a lot&mdash;such as this raw model output for a topic model I built on charter school websites:

<img src="../assets/topic_prevalence.png" alt="Topic model output" width="600" height="300"/>

I name the top topic "standards & assessment", the second "communication", the third "web administration", etc. What would you call the topic fourth from the top? The bottom one? 

What use is such a model? In addition to describing what topics/ideas are expressed in a text corpus, we can consider these topics' many contexts: race and class, emotions, popularity, etc. This lets us answer questions like: Which topics are less common in the websites of charter schools with a predominantly white student body? These regression results help us answer that particular question:

<img src="../assets/topics_and_race.png" alt="Topics and race" width="600" height="300"/>

We can also look at how topics change over time, producing visuals like this one, which visualizes the topics of Trump's tweets in 2020.

<img src="../assets/trump_topics.png" alt="Trump's most frequent topics" width="1000" height="500"/>

Here are some other examples of how topic modeling can provide insight into large corpora:

* Survey a collection that is too big to read closely, e.g. [Computational Historiography: Data Mining in a Century of Classics Journals](http://www.perseus.tufts.edu/publications/02-jocch-mimno.pdf) (PDF)
* Look at thematic trends over time in an archive, e.g. [Topic Modeling Martha Ballard's Diary](http://www.cameronblevins.org/posts/topic-modeling-martha-ballards-diary/)
* Create metadata for an archive to improve accessibility, e.g. [Topic modeling for the valorisation of digitised archives of the European Commission](https://ieeexplore.ieee.org/abstract/document/7840981)
* Understand current trends in social media relevant to your discipline, e.g. [Mining the Open Web with ‘Looted Heritage’](https://electricarchaeology.ca/2012/06/08/mining-the-open-web-with-looted-heritage-draft/)

### How LDA works (briefly) <a id='mechanics'></a>

We'll implement the original and (arguably) simplest topic modeling algorithm, Latent Dirichlet Allocation (or LDA), using Python's `scikit-learn`. There is a lot to learn about topic modeling, but the focus here is giving you starter code you can expand on later. To give a little more mathematical context, though, here are the mechanics of this version of topic modeling. 

LDA is a generative model&mdash;a model of the data-generating process&mdash;in which documents are mixtures of topics and topics are probability distributions over tokens in the vocabulary.$^1$ The (normalized) frequency of word $j$ in document $i$ can be written as:

$q_{ij} = v_{i1}*\theta_{1j} + v_{i2}*\theta_{2j} + ... + v_{iK}*\theta_{Kj}$

where K is the total number of topics, $\theta_{kj}$ is the probability that word $j$ shows up in topic $k$ and $v_{ik}$ is the weight assigned to topic $k$ in document $i$. The model treats $v$ and $\theta$ as generated from Dirichlet-distributed priors and estimates them with Maximum Likelihood or Bayesian methods.

To generate a document, LDA assumes that we choose a distribution over topics (what the text talks about); to generate each word in the document, we then choose a specific topic and&mdash;based on the distribution over words for that topic&mdash;choose a word. These mechanics can be illustrated as follows:$^2$

<img src="../assets/lda_blei.png" alt="The intuitions behind LDA (Blei 2012)" width="1000" height="500"/>

$^1$ Blei, D. M., A. Y. Ng, and M. I. Jordan (2003). Latent Dirichlet allocation. _Journal of Machine
Learning Research_ 3, 993–1022. <br/>
$^2$ Blei, D. M. (2003) Probabilistic Topic Models. _Communications of the ACM_ 55, 77–84.

---
---

## How to join in with coding <a id='joinin'></a>

* **Edit** any cell and try changing the code, or delete it and write your own.

* Before running a cell, think through the code and try to **guess** what output you'll get.

* If you encounter an **error**, take a breath; this is normal. Errors happen all the time, and by reading the error message you will often learn something new and useful.

* Remember: we are in the cloud, so you cannot break the notebook or your computer: **don't be afraid to experiment!**.

**Let's get coding!**

## The Pandas dataframe: Children's literature <a id='data'></a>

As our toy dataset, let's use a database of children's fictional literature from the 1880s. The data were compiled by students in [this course](http://english197s2015.pbworks.com/w/page/93127947/FrontPage) and have been minimally cleaned for our use. The data and other corpora can be [found here](http://dhresourcesforprojectbuilding.pbworks.com/w/page/69244469/Data%20Collections%20and%20Datasets#demo-corpora), feel free to explore that data and play with it using text analysis!

First, we read our corpus into a Pandas dataframe. The file is stored as a compressed .csv file in the `assets/` folder, so we tell Pandas the compression type (`.bz2`) so it can unpack and read it for us.

In [ ]:
import pandas as pd
import numpy as np
df_lit = pd.read_csv("../assets/childrens_lit.csv.bz2", 
                     sep='\t', 
                     index_col=0, 
                     encoding = 'utf-8', 
                     compression='bz2')

#drop rows where the text is missing.
df_lit = df_lit.dropna(subset=['text'])

#view the dataframe
df_lit

### Explore the data <a id='explore'></a>

Let's look at some descriptive statistics about this dataset, to get a feel for what's in it and check for any visible errors. This will help with interpreting the data and knowing what kinds of questions to ask. 

The first thing most people do is to `describe` their data. (This is the `summary` command in R, or the `sum` command in Stata).

In [ ]:
#There's only one numeric column in our data ('year'), so we only get one column for output.
df_lit.describe()

In [ ]:
#Visualize the distribution of the 'year' column
import matplotlib
df_lit['year'].hist(
    bins = 10, 
    grid=False)

In [ ]:
# To get summaries of some of non-numerical columns, we can explicitly ask for it.
df_lit.describe(include=['O'])

What's the breakdown of author genders in this dataset?

In [ ]:
#We can count this using the value_counts() function
df_lit['author gender'].value_counts()

What kind of titles do these books have?

In [ ]:
df_lit['title'].value_counts().head(10)

What is the average year for each author gender? To answer this question, we use the powerful `groupby` function in Pandas. (Similar to `collapse` on Stata)

In [ ]:
books_grouped_by_gender = df_lit.groupby("author gender")
books_grouped_by_gender['year'].mean().sort_values(ascending=False)

## Text preprocessing: The Document-Term Matrix (DTM) <a id='preprocess'></a>

>**The key to understanding Natural Language Processing (NLP) is that the computer can only do computations on _numbers_. We have to present our corpus for analysis in a numerical form&mdash;typically _vectors_&mdash;and make human sense of everything at the end.**

A "bag of words" corpus is the _vocabulary_ of tokens (words) in the corpus together with some _measure_ of how often they occur. The measurement may be:
* binary (presence or absence)
* count or frequency (how many times the token occurs)
* proportion (count divided by the total number of tokens).
* some other weighting scheme (more on this tomorrow!)

This is called a "bag of words" because the order and location of words is discarded. For example, it doesn't matter if the words 'red' and 'nose' are adjacent ('red nose'), or at the beginning or end of a sentence; this approach just treats the words individually. It's like a 'bag' of Scrabble™ tiles, where each tile is a word, all rattling around together in no particular order. It looks like this:
<br/><br/>
<img src="../assets/bag_of_words.png" alt="Bag of words" width="800" height="300"/>
<br/><br/>
To represent a corpus as a bag of words for topic modeling applications and the like, a common preprocessing step is to create a **Document-Term Matrix (DTM)**. A DTM is a mathematical matrix that describes the frequency of terms that occur in a collection of documents. In a DTM, rows correspond to documents in the collection and columns correspond to terms. In other words, each document becomes a _vector_ of word counts (or some other measure) with a value for each word in that document. 

To turn our texts into numbers in the form of a DTM, let's use the `CountVectorizer` function from `scikit-learn` ([more info here](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)). This is a standard method of using DTM in Python.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Vectorize our text using CountVectorizer
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.80, min_df=50,
                                max_features=None,
                                stop_words='english')

dtm = tf_vectorizer.fit_transform(df_lit.text)
print("Done!")

Let's take a look at the DTM we just made.

In [ ]:
print(dtm)

This format is called Compressed Sparse Format. It saves a lot of memory to store the DTM in this format, but it is difficult to look at for a human. Compressed Sparse Format stores a matrix that has lots of zeroes, and saves space by omitting the zeroes. Let's look at a sample to see what it looks like WITH the zeroes:

In [ ]:
dtm.toarray()[:10,:10]

How do we know what each number (row) indicates in Compressed Sparse Format? We can access the words themselves through the `CountVectorizer` function `get_feature_names_out()` (formerly `get_feature_names()`). 

In [ ]:
print(tf_vectorizer.get_feature_names_out()[::100]) # get every 100th word

To illustrate the techniques in this lesson, we will first convert this matrix back to a Pandas dataframe, a format that's more intuitive to work with. For larger datasets, you will have to use the Compressed Sparse Format. 

*Note*: In practice it is rare to transform a DTM into a Pandas dataframe, because of memory issues. If using this notebook as a reference, don't just copy and paste--think about what you're doing to avoid crashing your system!

In [ ]:
vocab = tf_vectorizer.get_feature_names_out() # save terms in vectorizer for later use 

long_dtm = pd.DataFrame(dtm.toarray(), 
                        columns=vocab, 
                        index=df_lit.index)
long_dtm.head()

**What can we do with a DTM?**

Because DTMs have a matrix format, we can perform any matrix algebra or vector manipulation on it, which enables some pretty exciting things (think vector space and Euclidean geometry). But, what do we lose when we represent text in this format? (Hint: Think "bag of words".)

For a toy example, we can use our long-format DTM to quickly identify the most frequent words.

In [ ]:
long_dtm.sum().sort_values(ascending=False).head(10)

### Challenge

* Print out the 10 most _infrequent_ words (rather than the most frequent words). You can look at the [Pandas documentation](http://pandas.pydata.org/pandas-docs/stable/api.html#api-dataframe-stats) for more information.
* Print the average number of times 10 most frequent and the 10 most infrequent words are used in a book.
* Print this out sorted from highest to lowest.

In [ ]:
# your code here

### Extra challenge

What are the most frequent words for male and female authors?

_Hints:_ First, join the long-format DTM with the main DataFrame (`pd.concat(axis=1)` works well for this). Then filter by gender. Once this is done, remove the columns from the original DF before you print frequent words.

In [ ]:
# your code here

## Train a topic model using LDA <a id='train'></a>

The first thing we need to do is 'train a model'. These words and concepts come from the (fashionable!) sphere of **machine learning**. A 'model' is a simplified representation of something in the real world that a computer can handle. In topic modeling, the model is a representation of the topics of the corpus.

We can say the computer acquires 'experience', or a simplified 'understanding' of what the corpus is about by creating a model of it. We have to train the computer in how to make its model by feeding it training data&mdash;in the case of topic modeling, this data is the text we are interested in.

Let's train the model using the `scikit-learn` function `LatentDirichletAllocation`. See [here](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) for more information about this function. 

> **❗Important Note: Topic models are not perfectly reproducible, so you may see different topics, words, and/or probabilities  from what others see. This means my examples may not match up exactly with what you see when you run the notebook.**

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

n_samples = 2000
n_topics = 4
n_top_words = 50

# This is a function to print out the top words for each topic in a pretty way.
# Don't worry too much about understanding every line of this code.
# Question: What's missing from this function?
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

    print()

In [ ]:
print("Fitting LDA model with tf features, "
      "n_samples=%d and n_topics=%d..."
      % (n_samples, n_topics))

# define the lda function, with desired options
# Check the documentation, linked above, to look through the options
lda = LDA(n_components=n_topics, 
          max_iter=20,
          learning_method='online',
          learning_offset=80.,
          total_samples=n_samples,
          random_state=0)

# fit the model
lda.fit(dtm)
print("Done!")

In [ ]:
# print the top words per topic, using the function defined above.
# Unlike R, which has a built-in function to print top words, we have to write our own for scikit-learn

print("\nTopics in LDA model:")
print_top_words(lda, vocab, n_top_words)

Take a minute to inspect these top words. _What name would you give each of these topics?_ (This is a common task in topic modeling.)

### Challenge

Copy and paste the above code and fit a new model, `lda_new`, by changing some of the parameters. How does this change the output?

Suggestions:
1. Change the number of topics. 
2. Do not remove stop words. 
3. Change other options, either in the vectorize stage or the LDA model.

_Hint:_ If you change the text vectorization method, don't forget to re-define your vocabulary for the `print_top_words()` function with code like this:
```python
vocab = tf_vectorizer.get_feature_names_out()
```

In [ ]:
# your code here

## Document-by-topic distribution <a id='topics'></a>

One thing we may want to do with a topic model is find the most representative texts for each topic. A simple way to do this (but not memory efficient) is to merge the topic distribution back into the Pandas dataframe.

Hopefully you've only been modifying `lda_new` and our original `lda` model is still intact (if not, re-run the LDA cells before the last challenge). Using our original model with four topics, English stopwords, etc., let's first get the topic distribution array.

In [ ]:
topic_dist = lda.transform(dtm)
topic_dist

Merge back in with the original dataframe:

In [ ]:
topic_dist_df = pd.DataFrame(topic_dist)
df_w_topics = topic_dist_df.join(df_lit)
df_w_topics

Now we can sort the dataframe for the topic of interest, and view the top documents for the topics.
Below we sort the documents first by Topic 0 (looking at the top words for this topic I think it's about family, health, and domestic activities), and next by Topic 1 (again looking at the top words I think this topic is about children playing outside in nature). These topics may be a family/nature split?

Look at the titles for the two different topics. Look at the gender of the author. Hypotheses?

In [ ]:
print(df_w_topics[['title', 'author gender', 0]].sort_values(by=[0], ascending=False))

In [ ]:
print(df_w_topics[['title', 'author gender', 1]].sort_values(by=[1], ascending=False))

### Challenge

- What is the average topic weight by author gender, for each topic?
- Which topic is most represented in texts by women? Most represented in texts by men?
- Which topic is least represented in texts by women? Least represented in texts by men?
- Graph these results.

Hint 1: Consider using the python `range` function and a for-loop to create a list of topic indices and inspect average topic weights. This code block gets that started for you.

In [ ]:
topic_columns = range(0,4)
for num in topic_columns:
    print(num)

Hint 2: Use a Pandas `groupby()` to compare the topic loadings of Male and Female authors.

In [ ]:
#your code here

## LDA as dimensionality reduction <a id='dimensionality'></a>

Another use for topic modeling is to reduce the dimensionality of text. Instead of representing a document as a vector of token counts over the whole vocabulary (which may be very large), topic modeling can represent that document as a weighted average of _k_ topics over a finite number of (more or less) interpretable topics. The latter approach is usually more tractable for subsequent statistical analysis (linear regression, decision trees, transformers, etc). 

Now that we obtained a distribution of topic weights for each document, we can represent our corpus with a dense document-weight matrix as opposed to our initial sparse DTM. The weights can then replace tokens as features for any subsequent task (classification, prediction, etc). A simple example may consist in measuring cosine similarity between documents. For instance, which book is closest to the first book in our corpus? Let's use pairwise cosine similarity to find out. 

NB: cosine similarity measures an angle between two vectors, which provides a measure of distance robust to vectors of different lenghts (total number of tokens)

First, let's create a sparse DTM.

In [ ]:
dtm = tf_vectorizer.fit_transform(df_lit['text'])

Next let's import the `cosine_similarity` function from `sklearn` and print the cosine similarity between the first and second book or the first and third book. Let's use the built-in `time` function to tell us how long this takes.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import time

start = time.time()
print("Cosine similarity between first and second book: ", 
      str(cosine_similarity(dtm[0,:], dtm[1,:])))
print("Cosine similarity between first and third book: ", 
      str(cosine_similarity(dtm[0,:], dtm[2,:])))
end = time.time()

print()
print("Computing the above code took " + str(end-start) + " seconds.")

What if we use the topic weights instead of word frequencies? Would this get similar results, and would it be faster? 

To try this out, it's more straightforward to calculate cosine similarity with the `spatial` function from `scipy`.

In [ ]:
from scipy import spatial

dwm = df_w_topics.iloc[:,:4] # limit docs/topics DF to only topic weights

start = time.time()
print("Cosine similarity between first and second book: ", 
      str(1-spatial.distance.cosine(dwm.iloc[0,:], dwm.iloc[1,:])))
print("Cosine similarity between first and third book: ", 
      str(1-spatial.distance.cosine(dwm.iloc[0,:], dwm.iloc[2,:])))
end = time.time()

print()
print("Computing the above code took " + str(end-start) + " seconds.")

It looks like using topic weights to compute similarities between a few documents took less than half the time than did using word frequencies. 

### Challenge

Calculate the cosine similarity between the first book and all other books. What is the most similar book to the first one?

_Hint 1:_ Use the `df.iterrows()` method to iterate over all rows in the document/topic weights DataFrame we just made. 

_Hint 2:_ To store the similarities, consider using a Python dictionary&mdashthat is, a list of key-value pairs. `max()` will get you the highest entry in the dictionary, and `dict.values()` will get you the dictionary values.

In [ ]:
# your code here